In [14]:
import requests
import defs
import pandas as pd

In [15]:
session = requests.Session()

In [16]:
ins_df = pd.read_pickle("instruments.pkl")

In [17]:
our_curr = ['EUR','USD','GBP','JPY','CHF','NZD','CAD']

In [18]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    
    params = dict(
    count = count,
    granularity = granularity,
    price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [19]:
code, res = fetch_candles("EUR_USD",9,"H1")

In [20]:
res

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 3605,
   'time': '2024-09-18T16:00:00.000000000Z',
   'bid': {'o': '1.11151', 'h': '1.11254', 'l': '1.11148', 'c': '1.11235'},
   'mid': {'o': '1.11158', 'h': '1.11262', 'l': '1.11155', 'c': '1.11242'},
   'ask': {'o': '1.11166', 'h': '1.11271', 'l': '1.11162', 'c': '1.11250'}},
  {'complete': True,
   'volume': 4340,
   'time': '2024-09-18T17:00:00.000000000Z',
   'bid': {'o': '1.11237', 'h': '1.11298', 'l': '1.11164', 'c': '1.11233'},
   'mid': {'o': '1.11244', 'h': '1.11306', 'l': '1.11171', 'c': '1.11270'},
   'ask': {'o': '1.11251', 'h': '1.11314', 'l': '1.11178', 'c': '1.11308'}},
  {'complete': True,
   'volume': 26382,
   'time': '2024-09-18T18:00:00.000000000Z',
   'bid': {'o': '1.11234', 'h': '1.11884', 'l': '1.11230', 'c': '1.11481'},
   'mid': {'o': '1.11271', 'h': '1.11892', 'l': '1.11268', 'c': '1.11489'},
   'ask': {'o': '1.11308', 'h': '1.11900', 'l': '1.11307', 'c': '1.11497'}}

In [21]:
code

200

In [22]:
def get_candles_df(json_response):
    
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)


In [23]:
get_candles_df(res)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-09-18T16:00:00.000000000Z,3605,1.11158,1.11262,1.11155,1.11242,1.11151,1.11254,1.11148,1.11235,1.11166,1.11271,1.11162,1.11250
1,2024-09-18T17:00:00.000000000Z,4340,1.11244,1.11306,1.11171,1.11270,1.11237,1.11298,1.11164,1.11233,1.11251,1.11314,1.11178,1.11308
2,2024-09-18T18:00:00.000000000Z,26382,1.11271,1.11892,1.11268,1.11489,1.11234,1.11884,1.11230,1.11481,1.11308,1.11900,1.11307,1.11497
3,2024-09-18T19:00:00.000000000Z,19376,1.11492,1.11492,1.10966,1.11057,1.11485,1.11485,1.10959,1.11049,1.11500,1.11500,1.10974,1.11065
4,2024-09-18T20:00:00.000000000Z,3573,1.11056,1.11205,1.11046,1.11177,1.11048,1.11198,1.11038,1.11169,1.11063,1.11213,1.11053,1.11185
5,2024-09-18T21:00:00.000000000Z,416,1.11164,1.11184,1.11146,1.11168,1.11122,1.11169,1.11110,1.11154,1.11207,1.11209,1.11161,1.11183
6,2024-09-18T22:00:00.000000000Z,1427,1.11177,1.11241,1.11150,1.11218,1.11156,1.11232,1.11141,1.11211,1.11198,1.11250,1.11160,1.11225
7,2024-09-18T23:00:00.000000000Z,1733,1.11220,1.11220,1.11112,1.11140,1.11213,1.11213,1.11103,1.11132,1.11227,1.11227,1.11119,1.11147


In [24]:
def save_file(candles_df,pair,granularity):
    candles_df.to_pickle(f'his_data/{pair}_{granularity}.pkl')

In [25]:
def create_data(pair,granularity):
    code ,json_data = fetch_candles(pair,4000,granularity)
    if code != 200:
        print(pair,"Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df,pair, granularity)

In [26]:
for p1 in our_curr:
    for p2 in our_curr:
        pair =  f"{p1}_{p2}"
        if pair in ins_df.name.unique():
          create_data(pair,"H1")

EUR_USD loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
USD_JPY loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
USD_CHF loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
USD_CAD loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
GBP_USD loaded 3999 candles from 2024-01-30T08:00:00.000000000Z to 2024-09-18T23:00:00.000000000Z
GBP_JPY loaded 3999 

In [27]:
ins_df.shape[0]

68

In [28]:
ins_df

,name,type,displayName,pipLocation,marginRate
0,USD_CNH,CURRENCY,USD/CNH,-4,0.05
1,NZD_CAD,CURRENCY,NZD/CAD,-4,0.03
2,EUR_CHF,CURRENCY,EUR/CHF,-4,0.04
3,SGD_CHF,CURRENCY,SGD/CHF,-4,0.05
4,EUR_CZK,CURRENCY,EUR/CZK,-4,0.05
...,...,...,...,...,...
63,USD_SEK,CURRENCY,USD/SEK,-4,0.03
64,GBP_SGD,CURRENCY,GBP/SGD,-4,0.05
65,EUR_PLN,CURRENCY,EUR/PLN,-4,0.05
66,HKD_JPY,CURRENCY,HKD/JPY,-4,0.1


In [29]:
ins_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   type         68 non-null     object
 2   displayName  68 non-null     object
 3   pipLocation  68 non-null     int64 
 4   marginRate   68 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.8+ KB


In [30]:
ins_df.columns


Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [31]:
ins_df.name

0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object

In [32]:
ins_df.name.unique

<bound method Series.unique of 0     USD_CNH
1     NZD_CAD
2     EUR_CHF
3     SGD_CHF
4     EUR_CZK
       ...   
63    USD_SEK
64    GBP_SGD
65    EUR_PLN
66    HKD_JPY
67    EUR_HUF
Name: name, Length: 68, dtype: object>